In [12]:
!pip install sentence_transformers inscriptis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 23.3 MB/s eta 0:00:0000:0100:01


In [13]:
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
from inscriptis import get_text

In [5]:
embedding_model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/112k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [8]:
dataset = load_dataset("mwarchalowski/grants", "no-shorts-no-dups")


In [31]:
def clean_html(examples):
    return {'abstr': [get_text(x) for x in examples['abstr']]} 

In [32]:
cleanDS = dataset.map(clean_html, batched=True)
cleanDS

Map:   0%|          | 0/126778 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'abstr', 'text_len'],
        num_rows: 126778
    })
})

In [33]:
def text_size(str):
    return len(str.split(" "))


In [34]:
cleanDS1 = cleanDS.map(lambda example: {'text_len':text_size(" ".join([example["title"], example["abstr"]]))})


Map:   0%|          | 0/126778 [00:00<?, ? examples/s]

In [35]:
cleanDS1

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'abstr', 'text_len'],
        num_rows: 126778
    })
})

In [36]:
embeddings = embedding_model.encode(cleanDS1["train"]["abstr"], show_progress_bar=True)

Batches:   0%|          | 0/3962 [00:00<?, ?it/s]

In [45]:
from safetensors.torch import save_file
import torch


In [46]:
save_file({'full': torch.from_numpy(embeddings)}, "/workspace/1/full_embeddings.safetensors")

In [ ]:
to_be_saved = cleanDS1.train_test_split(0

In [61]:
cleanDS1.push_to_hub("mwarchalowski/grants", "full_clean_emb_mapped", commit_message="Clean Embedded")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/127 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/829 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mwarchalowski/grants/commit/a8b587798e71a5f390786374d6869c9f8d92863c', commit_message='Clean Embedded', commit_description='', oid='a8b587798e71a5f390786374d6869c9f8d92863c', pr_url=None, pr_revision=None, pr_num=None)

In [59]:
from huggingface_hub import notebook_login

In [60]:
notebook_login()